### Transcript Loading

In [2]:
def read_text_file(file_path: str) -> str:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"file not here: {file_path}")
    except Exception as e:
        print(f"error: {file_path}")
    return ""

In [3]:
transcript=read_text_file("transcripts/rag.txt")

In [4]:
transcript[:100]

'Transcript: RAG Agentic Bot Requirements Call\n\nParticipants:\nRohit (Client, Head of Innovation at a '

### Text processing Layer

In [ ]:
import re

def clean_text_for_llm(text: str) -> str:
    if not text:
        return ""

    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = "".join(ch for ch in text if ch.isprintable() or ch in "\n\t")
    text = re.sub(r"\n{3,}", "\n\n", text)
    paragraphs = re.split(r"\n{2,}", text)
    paragraphs = [" ".join(p.splitlines()).strip() for p in paragraphs if p.strip()]
    cleaned = "\n\n".join(paragraphs)
    return cleaned.strip()

### LLM Call

In [ ]:
import openai

openai.api_key= ""

def call_llm(model: str, system_prompt: str, user_prompt: str, temperature: float) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": user_prompt},],
        temperature=temperature,
        stream=True)
    result=""
    for chunk in response:
        if 'choices' in chunk and chunk['choices'][0].get('delta'):
            content = chunk['choices'][0]['delta'].get(content)
            if content:
                print(content, end="", flush=True)
                result+=content

    return result

### Solution Architect Layer

In [ ]:
solution_architect_system_prompt="""
You are acting as a senior solution architect. 
I will give you a transcript of a client call discussing project requirements. 
Your job is to produce a clear, structured, and detailed requirement analysis that can directly feed into a proposal. 
Make it simple but comprehensive, so that each part can expand into proposal sections later.

Cover the following:

1. Business Context & Problem
- What is the client’s domain and pain points?
- What exactly do they want to achieve?

2. Functional Requirements
- List each requirement explicitly as features or modules.
- Separate "must-have" vs "good-to-have".

3. Non-Functional Requirements
- Security, scalability, reliability, compliance, cost considerations.

4. End-to-End Flow (Step by Step)
Explain the process like a journey:
User → Interface → Backend services → Databases → AI/ML modules → External integrations → Output.  
Show the sequence clearly, in numbered steps.

5. Recommended Tech Stack & Services
- Frontend frameworks
- Backend frameworks
- Databases (SQL/NoSQL/vector stores)
- AI/ML services (if relevant)
- Cloud provider + services (compute, storage, monitoring, CI/CD, networking)
- Third-party integrations (payments, notifications, APIs)

6. High-Level Architecture
- Suggest how components connect: frontend, API gateway, microservices, DB, AI engine, integrations.
- Mention best practices (e.g., event-driven, modular, multi-tenant).

7. Deployment & Infrastructure Strategy
- Cloud choice and region
- Containers or serverless
- CI/CD pipeline
- Monitoring, logging, and observability stack

8. Phased Implementation Roadmap
- Break project into realistic phases (MVP, Expansion, Advanced).
- Tie each phase to deliverables and business value.

9. Best Strategy
- State the *recommended approach* (e.g., RAG-based retrieval for AI, modular SaaS design, serverless for cost efficiency, etc.)
- Why this approach is best in terms of cost, scalability, and maintainability.
"""

In [ ]:
solution_architecture = call_llm("o-reasoning-1", solution_architect_system_prompt, solution_architect_user_prompt, 0.2)